# Initial Cleaning

In [ ]:
#Install required libraries
!pip install pymongo
!pip install ssh-pymongo

In [ ]:
#import pymongo to connect to db
from pymongo import UpdateOne
from ssh_pymongo import MongoSession
import pprint

In [ ]:
#Set host and remotely connect to mongodb
MONGO_HOST = "ip_address"
MONGO_USER = "username"
MONGO_PASS = "password"
MONGO_PORT = 443

session = MongoSession(MONGO_HOST,
    port=MONGO_PORT,
    user=MONGO_USER,
    password=MONGO_PASS,
    uri='mongodb:/'+MONGO_HOST+':'+str(MONGO_PORT)
)

Collections:

DIBBSAward2023-04-13

DIBBSAward2023-04-15

CICIAward2023-04-10

CSSIAward2023-04-16

MRIAward2023-04-16

CCFAward2023-04-11

SI2Award2023-04-24

In [ ]:
#List of collection names to iterate over as stored in the db

collection_names = ['DIBBSAward2023-04-13', 'DIBBSAward2023-04-15', 'CICIAward2023-04-10', 'CSSIAward2023-04-16', 'MRIAward2023-04-16', 'CCFAward2023-04-11', 'SI2Award2023-04-24']

In [ ]:
#Set variables for each respective db
local = session.connection['local']
clean = session.connection['tidy']

#Big regex string containing common words
nonowords = '(javascript)|(access denied)|(403.*error)|(403.*forbidden)|(can\'t find the page)|(404 not found)|(site under maintenance)|(down for maintenance)'

for curCollection in collection_names:
  print('Cleaning for ' + curCollection)
  print('-------------------')
  #set current collection to clean
  db = local[curCollection]
  print('Current # of documents: ' + str(db.count_documents({})))

  #get all NON empty documents and ones that DON'T match the regex
  getAllClean = list(db.find(
      {'$nor' : [{'raw html' : ''}, {'raw html' : { '$regex' : nonowords, '$options' : 'i'}}]}
  ))

  print('Documents after regex clean: ' + str(len(getAllClean)))

  #insert all found 'clean' documents into the clean db under the same collection name
  db_clean = clean[curCollection] 
  db_clean.insert_many(getAllClean)

  #delete documents that are too short (<400 chars)
  db_clean.delete_many({'$expr' : { '$lt': [{ '$strLenCP': '$raw html' }, 400]}})

  print('Documents after small truncation: ' + str(db_clean.count_documents({})) + '\n')


Cleaning for DIBBSAward2023-04-13
-------------------
Current # of documents: 13000
Documents after regex clean: 7949
Documents after small truncation: 5668

Cleaning for DIBBSAward2023-04-15
-------------------
Current # of documents: 26495
Documents after regex clean: 16649
Documents after small truncation: 11101

Cleaning for CICIAward2023-04-10
-------------------
Current # of documents: 28944
Documents after regex clean: 16486
Documents after small truncation: 11996

Cleaning for CSSIAward2023-04-16
-------------------
Current # of documents: 4000
Documents after regex clean: 2262
Documents after small truncation: 1621

Cleaning for MRIAward2023-04-16
-------------------
Current # of documents: 123115
Documents after regex clean: 72283
Documents after small truncation: 63542

Cleaning for CCFAward2023-04-11
-------------------
Current # of documents: 88334
Documents after regex clean: 52218
Documents after small truncation: 43609

Cleaning for SI2Award2023-04-24
------------------

# Sentiment Analysis and Tokenization


In [ ]:
#Importing essential tools
import os
import re
import pandas as pd
import gensim
import gensim.corpora as corpora
import nltk
import pickle 
import seaborn as sns;

from textblob import TextBlob
from bson.objectid import ObjectId

from wordcloud import WordCloud
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Installs
import sys
!{sys.executable} -m pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'https'])

In [ ]:
#Cleans the strings of unwanted characters
def cleanString(text):
  text = re.sub('[,/\.!?\n|\-#]', ' ', text)
  text = re.sub(r' +', ' ', text)
  return text

#Runs through sentiment analysis
def findSentiment(text):
  return TextBlob(text)
  
#Function removes punctuations from topics in long_string.
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

#Function removes stop_words from long_string, returns list of strings in long_string.
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]

def remove_stop_single(text):
    return [word for word in simple_preprocess(str(text)) 
             if word not in stop_words]

In [ ]:
#List of collection names to iterate over as stored in the db 
#'DIBBSAward2023-04-13', 'DIBBSAward2023-04-15', 'CICIAward2023-04-10', 'CSSIAward2023-04-16', 'MRIAward2023-04-16', 'CCFAward2023-04-11', 'SI2Award2023-04-24'

collection_names = ['CCFAward2023-04-11']

In [ ]:
clean = session.connection['tidy']

for curCollection in collection_names:
  print('Currently cleaning and tokenizing: ' + curCollection)
  print('-------------------')
  db_clean = clean[curCollection]

  data = db_clean.find({})
  df = pd.DataFrame(list(data))

  print('Starting clean html...')
  df['clean html'] = df['raw html'].apply(cleanString)
  print('Done with clean html!\n')

  print('Starting sentiment...')
  df['sentiment'] = df['clean html'].apply(lambda x : TextBlob(x).polarity)
  print('Done with sentiment!\n')

  print('Starting subjectivity...')
  df['subjectivity'] = df['clean html'].apply(lambda x : TextBlob(x).subjectivity)
  print('Done with subjectivity!\n')

  print('Writing clean, sentiment, and subjectivity...')
  data_dict = df.to_dict('records')
  updates = []
  for _, row in df.iterrows():
    updates.append( UpdateOne({'_id': row.get('_id')}, {'$set': {'sentiment': row.get('sentiment'), 'subjectivity': row.get('subjectivity'), 'clean html': row.get('clean html')}}, upsert=False))
  db_clean.bulk_write(updates)
  print('Done writing!\n')
  df = df.drop(['sentiment', 'subjectivity', 'raw html'], axis=1)

  print('Starting tokenization...')
  html_data = df['clean html'].tolist()
  html_iterator = sent_to_words(html_data)
  html_words = []
  for text in html_iterator:
    html_word = remove_stop_single(text)
    html_words.append(html_word)
  df['tokens'] = html_words
  print('Done with tokenization!\n')

  print('Writing all tokens...\n')
  data_dict = df.to_dict('records')
  updates = []
  for _, row in df.iterrows():
    updates.append( UpdateOne({'_id': row.get('_id')}, {'$set': {'tokens': row.get('tokens')}}, upsert=False))
  db_clean.bulk_write(updates)
  print('Done writing!\n')

  """
  data_dict = df.to_dict('records')
  updates = []
  for _, row in df.iterrows():
    updates.append( UpdateOne({'_id': row.get('_id')}, {'$set': {'sentiment': row.get('sentiment'), 'subjectivity': row.get('subjectivity'), 'clean html': row.get('clean html'), 'tokens' : row.get('tokens')}}, upsert=False))
  db_clean.bulk_write(updates)
  """

In [ ]:
for curCollection in collection_names:
  db_clean = db_clean = clean[curCollection]
  db_clean.delete_many({'$expr' : { '$lt': [{ '$strLenCP': '$clean html' }, 300]}})

# Testing on single database

In [ ]:
clean = session.connection['tidy']
db_clean = clean['CSSIAward2023-04-16']
data = db_clean.find({})

In [ ]:
df = pd.DataFrame(list(data))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1621 entries, 0 to 1620
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   _id       1621 non-null   object
 1   Award     1621 non-null   object
 2   keyword   1621 non-null   object
 3   title     1621 non-null   object
 4   text      1621 non-null   object
 5   raw html  1621 non-null   object
dtypes: object(6)
memory usage: 76.1+ KB


In [ ]:
def cleanString(text):
  text = re.sub('[,/\.!?\n|\-#]', '', text)
  text = re.sub(r' +', ' ', text)
  return text

In [ ]:
df['clean html'] = df['raw html'].apply(cleanString)

In [ ]:
df.head()

,_id,Award,keyword,title,text,raw html,clean html
0,6440840da8cdca1ec8c5e6f7,CSSI Award,agriculture,None,None,\n\n\n\n Cyberinfrastructure for Sustai...,Cyberinfrastructure for Sustained Scientific ...
1,6440840da8cdca1ec8c5e6fa,CSSI Award,agriculture,None,None,\n\n\n\n\n\n\n\nCSSI\n\n\n\n\n\n\n\n\n\n\n\n\n...,CSSI0 Skip to Content CSSI News Awards Fellow ...
2,6440840da8cdca1ec8c5e6fc,CSSI Award,agriculture,None,None,\n\n\n\n\nCSSI Inc. Receives Major NextGen Eng...,CSSI Inc Receives Major NextGen Engineering Co...
3,6440840da8cdca1ec8c5e6ff,CSSI Award,agriculture,None,None,\n\nFaculty & staff awards - Gustavson School ...,Faculty & staff awards Gustavson School of Bus...
4,6440840da8cdca1ec8c5e702,CSSI Award,agriculture,None,None,\n\n\n\n\n\n\n\n\n\n\n\nSECAF\n\n\n\n\n\n\n\n\...,SECAFJoin NowLoginMy AccountSECAF ChatContact ...


In [ ]:
def findSentiment(text):
  return TextBlob(text)

In [ ]:
df['textblob'] = df['clean html'].apply(lambda x : TextBlob(x))

In [ ]:
df['sentiment'] = df['textblob'].apply(lambda x : x.polarity)
df['subjectivity'] = df['textblob'].apply(lambda x : x.subjectivity)

In [ ]:
df.head()

,_id,Award,keyword,title,text,raw html,clean html,textblob,sentiment,subjectivity
0,6440840da8cdca1ec8c5e6f7,CSSI Award,agriculture,None,None,\n\n\n\n Cyberinfrastructure for Sustai...,Cyberinfrastructure for Sustained Scientific ...,"( , C, y, b, e, r, i, n, f, r, a, s, t, r, u, ...",0.061404,0.491228
1,6440840da8cdca1ec8c5e6fa,CSSI Award,agriculture,None,None,\n\n\n\n\n\n\n\nCSSI\n\n\n\n\n\n\n\n\n\n\n\n\n...,CSSI0 Skip to Content CSSI News Awards Fellow ...,"(C, S, S, I, 0, , S, k, i, p, , t, o, , C, ...",0.044834,0.330483
2,6440840da8cdca1ec8c5e6fc,CSSI Award,agriculture,None,None,\n\n\n\n\nCSSI Inc. Receives Major NextGen Eng...,CSSI Inc Receives Major NextGen Engineering Co...,"(C, S, S, I, , I, n, c, , R, e, c, e, i, v, ...",0.225415,0.586469
3,6440840da8cdca1ec8c5e6ff,CSSI Award,agriculture,None,None,\n\nFaculty & staff awards - Gustavson School ...,Faculty & staff awards Gustavson School of Bus...,"(F, a, c, u, l, t, y, , &, , s, t, a, f, f, ...",0.198893,0.401283
4,6440840da8cdca1ec8c5e702,CSSI Award,agriculture,None,None,\n\n\n\n\n\n\n\n\n\n\n\nSECAF\n\n\n\n\n\n\n\n\...,SECAFJoin NowLoginMy AccountSECAF ChatContact ...,"(S, E, C, A, F, J, o, i, n, , N, o, w, L, o, ...",0.201229,0.535187


In [ ]:
df['sentiment'] = df['clean html'].apply(lambda x : TextBlob(x).polarity)
df['subjectivity'] = df['clean html'].apply(lambda x : TextBlob(x).subjectivity)

In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'https'])

In [ ]:
#Function removes punctuations from topics in long_string.
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

#Function removes stop_words from long_string, returns list of strings in long_string.
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

In [ ]:
html_data = df['clean html'].tolist()
html_words = list(sent_to_words(html_data))
html_words = remove_stopwords(html_words)

In [ ]:
df['tokens'] = html_words

In [ ]:
df.head()

,_id,Award,keyword,title,text,raw html,clean html,tokens
0,6440840da8cdca1ec8c5e6f7,CSSI Award,agriculture,None,None,\n\n\n\n Cyberinfrastructure for Sustai...,Cyberinfrastructure for Sustained Scientific ...,"[sustained, scientific, innovation, cssi, la, ..."
1,6440840da8cdca1ec8c5e6fa,CSSI Award,agriculture,None,None,\n\n\n\n\n\n\n\nCSSI\n\n\n\n\n\n\n\n\n\n\n\n\n...,CSSI0 Skip to Content CSSI News Awards Fellow ...,"[cssi, skip, content, cssi, news, awards, fell..."
2,6440840da8cdca1ec8c5e6fc,CSSI Award,agriculture,None,None,\n\n\n\n\nCSSI Inc. Receives Major NextGen Eng...,CSSI Inc Receives Major NextGen Engineering Co...,"[cssi, inc, receives, major, nextgen, engineer..."
3,6440840da8cdca1ec8c5e6ff,CSSI Award,agriculture,None,None,\n\nFaculty & staff awards - Gustavson School ...,Faculty & staff awards Gustavson School of Bus...,"[faculty, staff, awards, gustavson, school, bu..."
4,6440840da8cdca1ec8c5e702,CSSI Award,agriculture,None,None,\n\n\n\n\n\n\n\n\n\n\n\nSECAF\n\n\n\n\n\n\n\n\...,SECAFJoin NowLoginMy AccountSECAF ChatContact ...,"[secafjoin, nowloginmy, accountsecaf, chatcont..."


In [ ]:
data_dict = df.to_dict('records')

In [ ]:
updates = []
for _, row in df.iterrows():
    updates.append( UpdateOne({'_id': row.get('_id')}, {'$set': {'sentiment': row.get('sentiment'), 'subjectivity': row.get('subjectivity'), 'clean html': row.get('clean html'), 'tokens' : row.get('tokens')}}, upsert=False))
    #updates.append( UpdateOne({'_id': row.get('_id')}, {'$set': {'sentiment': row.get('sentiment'), 'subjectivity': row.get('subjectivity'), 'clean html': row.get('clean html'), 'lemma tokens' : row.get('lemma_tokens')}}, upsert=False))

2023-04-21 05:35:50,103| ERROR   | Could not establish connection from local ('127.0.0.1', 38673) to remote ('127.0.0.1', 27017) side of the tunnel: open new channel ssh error: SSH session not active
ERROR:sshtunnel.SSHTunnelForwarder:Could not establish connection from local ('127.0.0.1', 38673) to remote ('127.0.0.1', 27017) side of the tunnel: open new channel ssh error: SSH session not active


In [ ]:
db_clean.bulk_write(updates)

In [ ]:
id = "64407ef1a64b00c43c4e4ba2"
test = db_clean.find({'_id' : ObjectId(id)})

test_df = pd.DataFrame(list(test))
print(test_df)

2023-04-21 05:54:09,408| ERROR   | Could not establish connection from local ('127.0.0.1', 38673) to remote ('127.0.0.1', 27017) side of the tunnel: open new channel ssh error: SSH session not active
ERROR:sshtunnel.SSHTunnelForwarder:Could not establish connection from local ('127.0.0.1', 38673) to remote ('127.0.0.1', 27017) side of the tunnel: open new channel ssh error: SSH session not active


                        _id       Award     keyword title  text  \
0  64407ef1a64b00c43c4e4ba2  CSSI Award  assessment  None  None   

                                            raw html  \
0  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPre-Award Info...   

                                          clean html  sentiment  subjectivity  \
0  Pre-Award Information - Research and Sponsored...   0.033929      0.231199   

                                              tokens  
0  [pre-award, information, research, sponsored, ...  


In [ ]:
id = "64407ef1a64b00c43c4e4ba1"
test = db_clean.find({'_id' : ObjectId(id)})

test_df = pd.DataFrame(list(test))
print(test_df)

                        _id       Award     keyword title  text  \
0  64407ef1a64b00c43c4e4ba1  CSSI Award  assessment  None  None   

                                            raw html  
0  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...  


In [ ]:
session.stop()